In [1]:
import theano
import theano.tensor as T
import lasagne
import numpy as np

from load_mnist import *
from batchify import *

import matplotlib.pyplot as plt

/home/aditya/anaconda3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
X_train, y_train, X_test, y_test = load_data()

In [3]:
X = T.tensor4('X')
y = T.ivector('y')

In [4]:
input_layer = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=X)
#channel = 1 as images are black-white
input_layer_drop = lasagne.layers.DropoutLayer(input_layer, p=0.2)
    
hidden_layer_1 = lasagne.layers.DenseLayer(input_layer_drop, 
                                           num_units=800, 
                                           nonlinearity=lasagne.nonlinearities.rectify, 
                                           W=lasagne.init.GlorotUniform()
                                          )
hidden_layer_1_drop = lasagne.layers.DropoutLayer(hidden_layer_1, p=0.5)
    
hidden_layer_2 = lasagne.layers.DenseLayer(hidden_layer_1_drop, 
                                           num_units=800, 
                                           nonlinearity=lasagne.nonlinearities.rectify, 
                                           W=lasagne.init.GlorotUniform()
                                          )
hidden_layer_2_drop = lasagne.layers.DropoutLayer(hidden_layer_2, p=0.5)
    
output_layer = lasagne.layers.DenseLayer(hidden_layer_2_drop, 
                                         num_units=10, 
                                         nonlinearity=lasagne.nonlinearities.softmax
                                        )
    
prediction = lasagne.layers.get_output(output_layer)
loss = lasagne.objectives.categorical_crossentropy(prediction, y)
l1_regularization = lasagne.regularization.regularize_layer_params_weighted(output_layer, l1)*1e-4
loss = loss + l1_penalty
loss = loss.mean()


params = lasagne.layers.get_all_params(output_layer, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)
#Used Stochastic Gradient Descent here without momentum
    
validation_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
#We disable dropout for validation
validation_loss = lasagne.objectives.categorical_crossentropy(validation_prediction, y)
validation_loss = validation_loss.mean()
    
validation_accuracy = T.mean(T.eq(T.argmax(validation_prediction, axis=1), y), 
                      dtype=theano.config.floatX)

train_mlp = theano.function([X, y], loss, updates=updates)
validation_mlp = theano.function([X, y], [validation_loss, validation_accuracy])
#No updates performed in validation_mlp

In [10]:
epochs = 1
batch_size = 500

training_loss_list = []
test_accuracy_list = []

for i in range(epochs):
    for batch in batchify(X_train, y_train, batch_size):
        X_batch, y_batch = batch
        training_loss_list.append(train_mlp(X_batch, y_batch))
    
    test_accuracy_list = []
    for batch in batchify(X_test, y_test, batch_size):
        X_batch, y_batch = batch
        _, test_accuracy = validation_mlp(X_batch, y_batch)
        test_accuracy_list.append(test_accuracy*100)